# Spark + Iceberg Queries - K2 Medallion Architecture

Query Bronze/Silver/Gold Iceberg tables using PySpark locally from Jupyter notebook.

## Quick Start

### 1. Environment Setup

**Install Java (Required for PySpark):**

PySpark requires Java 11 or Java 17. Check if Java is installed:

```bash
java -version
```

If not installed:

**macOS:**
```bash
# Using Homebrew
brew install openjdk@17

# Set JAVA_HOME (add to ~/.zshrc or ~/.bash_profile)
export JAVA_HOME=$(/usr/libexec/java_home -v 17)
export PATH=$JAVA_HOME/bin:$PATH
```

**Ubuntu/Debian:**
```bash
sudo apt-get update
sudo apt-get install openjdk-17-jdk

# Set JAVA_HOME (add to ~/.bashrc)
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export PATH=$JAVA_HOME/bin:$PATH
```

**Verify Java installation:**
```bash
echo $JAVA_HOME
java -version  # Should show version 17.x.x
```

**Install Python dependencies:**
```bash
# Install Jupyter and PySpark (if not already installed)
cd /path/to/k2-market-data-platform
uv add jupyter pyspark==3.5.0

# Verify installation
uv run python -c "import pyspark; print(f'PySpark {pyspark.__version__}')"
```

### 2. Start Required Services

**Start all infrastructure:**
```bash
# From project root
docker-compose up -d

# Verify services are running
docker-compose ps

# Should see:
# - k2-spark-master (port 8090 - Spark Web UI)
# - k2-spark-worker-1
# - k2-spark-worker-2  
# - k2-iceberg-rest (port 8181)
# - k2-minio (port 9000)
# - k2-kafka (port 9092)
# - k2-kafka-ui (port 8080)
```

**Check Spark Web UI:**
- Open: http://localhost:8090
- Verify: 2 workers registered (each with 2 cores, 3GB memory)

### 3. Create Bronze/Silver/Gold Tables

**Create Bronze tables (Binance + Kraken):**
```bash
docker exec k2-spark-master /opt/spark/bin/spark-submit \
  --master spark://spark-master:7077 \
  --jars /opt/spark/jars-extra/iceberg-spark-runtime-3.5_2.12-1.4.0.jar,/opt/spark/jars-extra/iceberg-aws-1.4.0.jar,/opt/spark/jars-extra/bundle-2.20.18.jar,/opt/spark/jars-extra/url-connection-client-2.20.18.jar,/opt/spark/jars-extra/hadoop-aws-3.3.4.jar \
  /opt/k2/src/k2/spark/jobs/create_bronze_table.py all
```

**Create Silver tables:**
```bash
docker exec k2-spark-master /opt/spark/bin/spark-submit \
  --master spark://spark-master:7077 \
  --jars /opt/spark/jars-extra/iceberg-spark-runtime-3.5_2.12-1.4.0.jar,/opt/spark/jars-extra/iceberg-aws-1.4.0.jar,/opt/spark/jars-extra/bundle-2.20.18.jar,/opt/spark/jars-extra/url-connection-client-2.20.18.jar,/opt/spark/jars-extra/hadoop-aws-3.3.4.jar \
  /opt/k2/src/k2/spark/jobs/create_silver_tables.py
```

**Create Gold table:**
```bash
docker exec k2-spark-master /opt/spark/bin/spark-submit \
  --master spark://spark-master:7077 \
  --jars /opt/spark/jars-extra/iceberg-spark-runtime-3.5_2.12-1.4.0.jar,/opt/spark/jars-extra/iceberg-aws-1.4.0.jar,/opt/spark/jars-extra/bundle-2.20.18.jar,/opt/spark/jars-extra/url-connection-client-2.20.18.jar,/opt/spark/jars-extra/hadoop-aws-3.3.4.jar \
  /opt/k2/src/k2/spark/jobs/create_gold_table.py
```

### 4. Start Data Producers (Optional)

**Start crypto streaming producers to populate Bronze tables:**
```bash
# Terminal 1 - Binance WebSocket
uv run python scripts/binance_stream.py

# Terminal 2 - Kraken WebSocket  
uv run python scripts/kraken_stream.py
```

Wait 30-60 seconds for data to accumulate in Kafka, then Bronze tables will be populated by the Spark streaming jobs.

### 5. Launch This Notebook

```bash
# Set JAVA_HOME first (if not in your shell profile)
export JAVA_HOME=$(/usr/libexec/java_home -v 17)  # macOS
# or
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64  # Linux

# Launch notebook from project root
uv run jupyter notebook demos/notebooks/spark-iceberg-queries.ipynb
```

## Architecture Overview

**Local (Notebook) → Docker Services:**
- **Notebook runs on host** with local PySpark installation
- **Connects to Docker services:**
  - Iceberg REST Catalog: `http://localhost:8181`
  - MinIO (S3): `http://localhost:9000`
  - Spark Cluster: Workers run in Docker (for streaming jobs)

**Data Flow:**
```
WebSocket → Kafka → Bronze (Spark Streaming) → Silver → Gold
                        ↓                         ↓       ↓
                    (This notebook queries these tables)
```

## Important Notes

1. **Java Required:** PySpark needs Java 11 or 17 installed locally
   - Check: `java -version`
   - Set `JAVA_HOME` environment variable

2. **Network Access:** All services must be accessible from host:
   - Spark Web UI: `localhost:8090`
   - Kafka UI: `localhost:8080`
   - Iceberg REST: `localhost:8181`
   - MinIO: `localhost:9000`
   - Kafka: `localhost:9092`

3. **Memory:** Ensure Docker has at least 8GB RAM allocated for Spark workers

## Troubleshooting

**Issue: "JAVA_HOME is not set" or "Java gateway process exited"**

```bash
# Check if Java is installed
java -version

# If not installed, install Java 17 (see step 1 above)

# Set JAVA_HOME
# macOS:
export JAVA_HOME=$(/usr/libexec/java_home -v 17)

# Linux:
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64

# Verify
echo $JAVA_HOME

# Add to shell profile for persistence
# macOS: ~/.zshrc or ~/.bash_profile
# Linux: ~/.bashrc
echo 'export JAVA_HOME=$(/usr/libexec/java_home -v 17)' >> ~/.zshrc  # macOS
```

**Issue: Cannot connect to Iceberg catalog**
```bash
# Check iceberg-rest is running
docker ps | grep iceberg-rest
curl http://localhost:8181/v1/config
```

**Issue: Cannot read from S3/MinIO**
```bash
# Check MinIO is running
docker ps | grep minio
# Access MinIO console: http://localhost:9001 (admin/password)
```

**Issue: Tables don't exist**
```bash
# Run table creation scripts (see step 3 above)
# Verify in MinIO: http://localhost:9001 → warehouse bucket
```

**Issue: Tables are empty**
```bash
# Check if producers are running and sending data to Kafka
docker exec k2-kafka kafka-console-consumer --bootstrap-server localhost:9092 \
  --topic market.crypto.trades.binance --max-messages 5

# Check if Bronze streaming jobs are running
docker exec k2-spark-master /opt/spark/bin/spark-submit \
  --master spark://spark-master:7077 \
  --jars /opt/spark/jars-extra/iceberg-spark-runtime-3.5_2.12-1.4.0.jar,/opt/spark/jars-extra/iceberg-aws-1.4.0.jar,/opt/spark/jars-extra/bundle-2.20.18.jar,/opt/spark/jars-extra/url-connection-client-2.20.18.jar,/opt/spark/jars-extra/hadoop-aws-3.3.4.jar,/opt/spark/jars-extra/spark-sql-kafka-0-10_2.12-3.5.3.jar,/opt/spark/jars-extra/kafka-clients-3.5.1.jar,/opt/spark/jars-extra/commons-pool2-2.11.1.jar,/opt/spark/jars-extra/spark-token-provider-kafka-0-10_2.12-3.5.3.jar \
  /opt/k2/src/k2/spark/jobs/streaming/bronze_binance_ingestion.py
```

---

**Ready to query? Execute the cells below sequentially.**

## Connect to Iceberg Catalog

**Note:** First run will download required JARs (~500MB) from Maven Central. This takes 2-3 minutes but is cached for future runs.

**Required JARs:**
- `iceberg-spark-runtime-3.5_2.12:1.4.0` - Iceberg integration
- `hadoop-aws:3.3.4` - S3A filesystem support  
- `software.amazon.awssdk:bundle:2.20.18` - AWS SDK v2 for S3/MinIO
- `software.amazon.awssdk:url-connection-client:2.20.18` - AWS HTTP client

In [1]:
from pyspark.sql import SparkSession

# Create Spark session with Iceberg catalog
# Note: Using .packages to automatically download required JARs
spark = (
    SparkSession.builder
    .appName("K2-Iceberg-Query-Demo")
    # Iceberg catalog configuration
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.iceberg.type", "rest")
    .config("spark.sql.catalog.iceberg.uri", "http://localhost:8181")  # iceberg-rest
    .config("spark.sql.catalog.iceberg.warehouse", "s3://warehouse/")
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://localhost:9000")  # minio
    .config("spark.sql.catalog.iceberg.s3.access-key-id", "admin")
    .config("spark.sql.catalog.iceberg.s3.secret-access-key", "password")
    .config("spark.sql.catalog.iceberg.s3.path-style-access", "true")
    # S3/MinIO configuration
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "password")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .config("spark.hadoop.fs.s3a.endpoint.region", "us-east-1")
    # AWS SDK v2 region configuration (required for MinIO)
    .config("spark.driver.extraJavaOptions", "-Daws.region=us-east-1")
    .config("spark.executor.extraJavaOptions", "-Daws.region=us-east-1")
    # Download required JARs automatically (Maven coordinates)
    .config("spark.jars.packages", 
            "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.0,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "software.amazon.awssdk:bundle:2.20.18,"
            "software.amazon.awssdk:url-connection-client:2.20.18")
    .getOrCreate()
)

print(f"✓ Spark {spark.version} connected to Iceberg catalog")

26/01/20 12:57:50 WARN Utils: Your hostname, RS-Dev-01 resolves to a loopback address: 127.0.1.1; using 192.168.4.25 instead (on interface wlo1)
26/01/20 12:57:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/rjdscott/.ivy2/cache
The jars for the packages stored in: /home/rjdscott/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-17847712-6f36-4b66-9ce7-d5f6dcb5e114;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.0 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central


:: loading settings :: url = jar:file:/home/rjdscott/Documents/projects/k2-market-data-platform/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found software.amazon.awssdk#bundle;2.20.18 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.20.18 in central
	found software.amazon.awssdk#utils;2.20.18 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found software.amazon.awssdk#annotations;2.20.18 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found software.amazon.awssdk#http-client-spi;2.20.18 in central
	found software.amazon.awssdk#metrics-spi;2.20.18 in central
:: resolution report :: resolve 148ms :: artifacts dl 6ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.0 from central in [default]
	org.reactivestreams#reactive-streams;1.0.3 from central in [default]
	org.slf4j#slf4j-api;1.7.30 from

✓ Spark 3.5.0 connected to Iceberg catalog


## List All Tables

In [2]:
## Verify Connection

# Check Spark version and configuration
print(f"Spark Version: {spark.version}")
print(f"Iceberg Catalog URI: {spark.conf.get('spark.sql.catalog.iceberg.uri')}")
print(f"Warehouse: {spark.conf.get('spark.sql.catalog.iceberg.warehouse')}")
print(f"S3 Endpoint: {spark.conf.get('spark.hadoop.fs.s3a.endpoint')}")

# Test catalog connection
try:
    spark.sql("SHOW NAMESPACES IN iceberg").show()
    print("\n✓ Successfully connected to Iceberg catalog")
except Exception as e:
    print(f"\n✗ Failed to connect to Iceberg catalog: {e}")

Spark Version: 3.5.0
Iceberg Catalog URI: http://localhost:8181
Warehouse: s3://warehouse/
S3 Endpoint: http://localhost:9000
+-----------+
|  namespace|
+-----------+
|market_data|
+-----------+


✓ Successfully connected to Iceberg catalog


In [3]:
spark.sql("SHOW TABLES IN iceberg.market_data").show(truncate=False)

+-----------+---------------------+-----------+
|namespace  |tableName            |isTemporary|
+-----------+---------------------+-----------+
|market_data|silver_kraken_trades |false      |
|market_data|silver_binance_trades|false      |
|market_data|gold_crypto_trades   |false      |
|market_data|silver_dlq_trades    |false      |
|market_data|bronze_kraken_trades |false      |
|market_data|bronze_binance_trades|false      |
+-----------+---------------------+-----------+



## Bronze Layer - Raw Kafka Data

Raw Avro bytes from Kafka (per-exchange tables).

**What to expect:**
- `bronze_binance_trades`: Raw Kafka messages from Binance WebSocket
- `bronze_kraken_trades`: Raw Kafka messages from Kraken WebSocket
- Fields: `message_key`, `avro_payload` (binary), Kafka metadata, ingestion timestamp
- Partitioned by: `ingestion_date`

**Note:** If tables are empty, start the Bronze streaming jobs and data producers (see setup instructions above).

**Quick check - see if any data exists:**

In [4]:
# Quick check - row counts for Bronze tables
print("Bronze Table Row Counts:")
print("=" * 50)

binance_count = spark.sql("SELECT COUNT(*) as count FROM iceberg.market_data.bronze_binance_trades").collect()[0]['count']
print(f"bronze_binance_trades: {binance_count:,} records")

kraken_count = spark.sql("SELECT COUNT(*) as count FROM iceberg.market_data.bronze_kraken_trades").collect()[0]['count']
print(f"bronze_kraken_trades: {kraken_count:,} records")

if binance_count == 0 and kraken_count == 0:
    print("\n⚠️  Bronze tables are empty. Start the Bronze streaming jobs and producers.")
else:
    print(f"\n✓ Found {binance_count + kraken_count:,} total records")

Bronze Table Row Counts:


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


bronze_binance_trades: 999,120 records
bronze_kraken_trades: 3,977 records

✓ Found 1,003,097 total records


## Bronze Layer - Raw Kafka Data

Raw Avro bytes from Kafka (per-exchange tables).

In [5]:
# Binance Bronze - row count by partition
spark.sql("""
    SELECT 
        ingestion_date,
        COUNT(*) as records,
        COUNT(DISTINCT partition) as kafka_partitions,
        MIN(kafka_timestamp) as first_trade,
        MAX(kafka_timestamp) as last_trade
    FROM iceberg.market_data.bronze_binance_trades
    GROUP BY ingestion_date
    ORDER BY ingestion_date DESC
""").show()

[Stage 6:=================================================>    (204 + 20) / 224]

+--------------+-------+----------------+--------------------+--------------------+
|ingestion_date|records|kafka_partitions|         first_trade|          last_trade|
+--------------+-------+----------------+--------------------+--------------------+
|    2026-01-20| 146511|               2|2026-01-20 12:19:...|2026-01-20 12:59:...|
|    2026-01-19| 809266|               2|2026-01-19 19:18:...|2026-01-20 10:42:...|
+--------------+-------+----------------+--------------------+--------------------+



In [7]:
# Binance Bronze - row count by partition
spark.sql("""
    SELECT * FROM iceberg.market_data.bronze_binance_trades
    limit 10
""").show()

+--------------------+--------------------+---------+------+--------------------+--------------------+--------------+
|           raw_bytes|               topic|partition|offset|     kafka_timestamp| ingestion_timestamp|ingestion_date|
+--------------------+--------------------+---------+------+--------------------+--------------------+--------------+
|[00 00 00 00 0A 0...|market.crypto.tra...|        0|670149|2026-01-19 23:14:...|2026-01-19 23:14:...|    2026-01-19|
|[00 00 00 00 0A 0...|market.crypto.tra...|        0|670150|2026-01-19 23:14:...|2026-01-19 23:14:...|    2026-01-19|
|[00 00 00 00 0A 0...|market.crypto.tra...|        0|670151|2026-01-19 23:14:...|2026-01-19 23:14:...|    2026-01-19|
|[00 00 00 00 0A 0...|market.crypto.tra...|        0|670152|2026-01-19 23:14:...|2026-01-19 23:14:...|    2026-01-19|
|[00 00 00 00 0A 0...|market.crypto.tra...|        0|670153|2026-01-19 23:14:...|2026-01-19 23:14:...|    2026-01-19|
|[00 00 00 00 0A 0...|market.crypto.tra...|        0|670

In [8]:
# Binance Bronze - row count by partition
spark.sql("""
    SELECT * FROM iceberg.market_data.bronze_kraken_trades
    limit 10
""").show()

+--------------------+--------------------+---------+------+--------------------+--------------------+--------------+
|           raw_bytes|               topic|partition|offset|     kafka_timestamp| ingestion_timestamp|ingestion_date|
+--------------------+--------------------+---------+------+--------------------+--------------------+--------------+
|[00 00 00 00 0B 8...|market.crypto.tra...|        0|   722|2026-01-19 23:09:...|2026-01-19 23:10:...|    2026-01-19|
|[00 00 00 00 0B 8...|market.crypto.tra...|        0|   723|2026-01-19 23:09:...|2026-01-19 23:10:...|    2026-01-19|
|[00 00 00 00 0B 8...|market.crypto.tra...|        5|   974|2026-01-19 23:09:...|2026-01-19 23:10:...|    2026-01-19|
|[00 00 00 00 0B 8...|market.crypto.tra...|        5|   975|2026-01-19 23:09:...|2026-01-19 23:10:...|    2026-01-19|
|[00 00 00 00 0B 8...|market.crypto.tra...|        5|   976|2026-01-19 23:09:...|2026-01-19 23:10:...|    2026-01-19|
|[00 00 00 00 0B 8...|market.crypto.tra...|        5|   

In [9]:
# Kraken Bronze - row count
spark.sql("""
    SELECT 
        ingestion_date,
        COUNT(*) as records,
        COUNT(DISTINCT partition) as kafka_partitions,
        MIN(kafka_timestamp) as first_trade,
        MAX(kafka_timestamp) as last_trade
    FROM iceberg.market_data.bronze_kraken_trades
    GROUP BY ingestion_date
    ORDER BY ingestion_date DESC
""").show()

+--------------+-------+----------------+--------------------+--------------------+
|ingestion_date|records|kafka_partitions|         first_trade|          last_trade|
+--------------+-------+----------------+--------------------+--------------------+
|    2026-01-19|   1493|               2|2026-01-19 19:18:...|2026-01-19 23:15:...|
+--------------+-------+----------------+--------------------+--------------------+



## Silver Layer - Validated Per-Exchange Trades

Deserialized and validated V2 trades (per-exchange).

In [6]:
# Binance Silver - recent trades
spark.sql("""
    SELECT * --timestamp, symbol, price, quantity, currency
    FROM iceberg.market_data.silver_binance_trades
    LIMIT 10
""").show(truncate=False)

+------------------------------------+------------------+-------+--------+-----------+----------------+-------------+----------+--------+----+----------------+---------------+-------------------+-----------------+------------------------------------------------------------------------------------------------+-----------------------+--------------------------+---------+
|message_id                          |trade_id          |symbol |exchange|asset_class|timestamp       |price        |quantity  |currency|side|trade_conditions|source_sequence|ingestion_timestamp|platform_sequence|vendor_data                                                                                     |validation_timestamp   |bronze_ingestion_timestamp|schema_id|
+------------------------------------+------------------+-------+--------+-----------+----------------+-------------+----------+--------+----+----------------+---------------+-------------------+-----------------+-------------------------------------------

In [16]:
spark.sql("""
    SELECT distinct symbol
    FROM iceberg.market_data.silver_binance_trades
    LIMIT 10
""").show(truncate=False)

+-------+
|symbol |
+-------+
|BTCUSDT|
|ETHUSDT|
+-------+



In [19]:
# Kraken Silver - recent trades
spark.sql("""
    SELECT * --timestamp, symbol, price, quantity, currency
    FROM iceberg.market_data.silver_kraken_trades
    LIMIT 10
""").show(truncate=False)

+------------------------------------+---------------------------------+------+--------+-----------+----------------+--------------+----------+--------+----+----------------+---------------+-------------------+-----------------+--------------------------------------------------------------------+-----------------------+--------------------------+---------+
|message_id                          |trade_id                         |symbol|exchange|asset_class|timestamp       |price         |quantity  |currency|side|trade_conditions|source_sequence|ingestion_timestamp|platform_sequence|vendor_data                                                         |validation_timestamp   |bronze_ingestion_timestamp|schema_id|
+------------------------------------+---------------------------------+------+--------+-----------+----------------+--------------+----------+--------+----+----------------+---------------+-------------------+-----------------+------------------------------------------------------

In [13]:
# Check Gold Trades
spark.sql("""
    SELECT 
    distinct exchange, count(exchange) --* --timestamp, symbol, price, quantity, currency
    FROM iceberg.market_data.gold_crypto_trades
    GROUP BY EXCHANGE
    LIMIT 10
""").show(truncate=False)

+--------+---------------+
|exchange|count(exchange)|
+--------+---------------+
|KRAKEN  |8187           |
|BINANCE |1860308        |
+--------+---------------+



In [ ]:
# Binance Silver - price aggregation (1-minute bars)
spark.sql("""
    SELECT 
        symbol,
        DATE_TRUNC('minute', CAST(timestamp / 1000000 AS TIMESTAMP)) as minute,
        COUNT(*) as trades,
        ROUND(AVG(price), 2) as avg_price,
        ROUND(MIN(price), 2) as low,
        ROUND(MAX(price), 2) as high,
        ROUND(SUM(quantity), 8) as volume
    FROM iceberg.market_data.silver_binance_trades
    WHERE symbol = 'BTCUSDT'
      AND exchange_date >= CURRENT_DATE() - INTERVAL 1 DAY
    GROUP BY symbol, minute
    ORDER BY minute DESC
    LIMIT 20
""").show()

In [ ]:
# Kraken Silver - recent trades
spark.sql("""
    SELECT 
        symbol,
        CAST(timestamp / 1000000 AS TIMESTAMP) as trade_time,
        price,
        quantity,
        side,
        vendor_data
    FROM iceberg.market_data.silver_kraken_trades
    WHERE exchange_date >= CURRENT_DATE() - INTERVAL 1 DAY
    ORDER BY timestamp DESC
    LIMIT 10
""").show(truncate=False)

## Gold Layer - Unified Multi-Exchange Analytics

Combined trades from all exchanges.

In [ ]:
# Gold - trades by exchange (last 24h)
spark.sql("""
    SELECT 
        exchange,
        COUNT(*) as trades,
        COUNT(DISTINCT symbol) as symbols,
        ROUND(SUM(price * quantity), 2) as total_value_usd
    FROM iceberg.market_data.gold_crypto_trades
    WHERE exchange_date >= CURRENT_DATE() - INTERVAL 1 DAY
    GROUP BY exchange
    ORDER BY trades DESC
""").show()

In [ ]:
# Gold - cross-exchange price comparison (BTC/USDT)
spark.sql("""
    SELECT 
        exchange,
        symbol,
        COUNT(*) as trades,
        ROUND(AVG(price), 2) as avg_price,
        ROUND(MIN(price), 2) as min_price,
        ROUND(MAX(price), 2) as max_price,
        ROUND(STDDEV(price), 2) as price_stddev
    FROM iceberg.market_data.gold_crypto_trades
    WHERE symbol IN ('BTCUSDT', 'BTCUSD')
      AND exchange_date >= CURRENT_DATE() - INTERVAL 1 DAY
    GROUP BY exchange, symbol
    ORDER BY exchange
""").show()

In [ ]:
# Gold - hourly trade volume (all exchanges)
spark.sql("""
    SELECT 
        exchange_date,
        exchange_hour,
        exchange,
        COUNT(*) as trades,
        ROUND(SUM(quantity), 8) as total_quantity
    FROM iceberg.market_data.gold_crypto_trades
    WHERE exchange_date >= CURRENT_DATE() - INTERVAL 1 DAY
    GROUP BY exchange_date, exchange_hour, exchange
    ORDER BY exchange_date DESC, exchange_hour DESC
    LIMIT 50
""").show()

## Table Schema Inspection

In [ ]:
# Describe Bronze table schema
spark.sql("DESCRIBE EXTENDED iceberg.market_data.bronze_binance_trades").show(100, truncate=False)

In [ ]:
# Describe Silver table schema
spark.sql("DESCRIBE EXTENDED iceberg.market_data.silver_binance_trades").show(100, truncate=False)

In [ ]:
# Describe Gold table schema
spark.sql("DESCRIBE EXTENDED iceberg.market_data.gold_crypto_trades").show(100, truncate=False)

## Table History & Snapshots

In [ ]:
# Show table history (Iceberg time-travel)
spark.sql("SELECT * FROM iceberg.market_data.bronze_binance_trades.history").show(truncate=False)

In [ ]:
# Show snapshots
spark.sql("SELECT * FROM iceberg.market_data.bronze_binance_trades.snapshots").show(truncate=False)

## Cleanup

In [ ]:
spark.stop()
print("✓ Spark session closed")